In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

import sys
sys.stdout = open('/dev/stdout', 'w')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)

def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [ ]:
path_to_train = "../data/train"

path1 = '../cache/train_hits.csv'
delete_file_if_exists(path1)

path2 = '../cache/train_truth.csv'
delete_file_if_exists(path2)


index = 0   

for event_id, hits, cells, particles, truth in tqdm(load_dataset(path_to_train)):

#     if index > 2:
#         continue
    truth1 = pd.merge(truth, particles, how='left', on='particle_id')
    hits1 = pd.merge(hits, truth1, how='left', on='hit_id')

    hits2 = hits1.dropna()
    truth2 = truth1.dropna()

    hits3 = hits2[hits2.nhits > 3]
    truth3 = truth2[truth2.nhits > 3]

    # new particle_id
    hits3['npid'] = hits3.particle_id.astype('str') + '_' + str(event_id)
    truth3['npid'] = truth3.particle_id.astype('str') + '_' + str(event_id)
#     train_hits = train_hits.append(hits3)
#     model.fit(hits3)
#     h_str = '{},{},{},{}'.format(hits3['hit_id'], hits3['x'], hits3['y'], hits3['z'])
#     f_sub.write(h_str)
    append_to_csv(hits3, path1)
    append_to_csv(truth3, path2)
    del hits1, truth1
    del hits2, truth2
    del hits3, truth3
    